In [1]:
!pip3 install scipy
!pip3 install tenacity
!pip3 install tiktoken
!pip3 install termcolor
!pip3 install openai
!pip3 install requests

In [5]:
import json
import os
import openai
from termcolor import colored

GPT_ENGINE = "nicogpt0631"

openai.api_type = "azure"
openai.api_base = "https://nico-openai.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
# READ KEY FROM ENVIRONMENT VARIABLE
openai.api_key = os.environ["OPENAI_API_KEY"]


9f9d6028d104404c810f7f91c5a952ca


# OpenAI Bank

This sample provides a high level pattern using Azure openAI `gpt-35-turbo` model, deployed using version `0613` to customize an AI chatbot banking assistant. The assistant focuses on helping customer with basic account management tasks including:
* Opening a new account

* List all accounts of the user
* Getting an account balance
* Making deposits
* Making transfers between accounts

It leverages the use of `functions` that help extend the model allowing to consume a set of available banking API functionality to fulfil users requests.

## Bank API

First we define our banking functionality. In this case using `python` methods that use a in memory account list.

In [26]:
# Accounts Database
accounts = []

In [27]:
# BANK API FUNCTIONS

import random

def create_bank_account(account_type, account_balance, user_id):
    # create an account with the given type and balance. Create a random account number and return it.

    account_number = random.randint(1000000000, 9999999999)

    # create a new account
    new_account = {
        "account_number": account_number,
        "account_type": account_type,
        "account_balance": account_balance,
        "user_id": user_id
    }

    accounts.append(new_account)

    return new_account

def get_user_accounts(user_id):

    # return a list of accounts that belong to the given user_id.
    return [account for account in accounts if account["user_id"] == user_id]

def get_account_balance(account_number):

    # return the account balance for the given account_number.
    for account in accounts:
        if account["account_number"] == int(account_number):
            return account["account_balance"]

    return None

def make_deposit(account_number, amount):
    print("make_deposit")
    print(account_number, amount)

    # make a deposit into the account with the given account_number. Return the new balance.
    for account in accounts:
        if account["account_number"] ==  int(account_number):
            account["account_balance"] += amount
            return account["account_balance"]

    return None

def make_transfer_between_accounts(from_account_number, to_account_number, amount):
    print("make_transfer_between_accounts")
    print(from_account_number, to_account_number, amount)
    
    # make a transfer between the two accounts. Return the new balance of the from_account.
    for account in accounts:
        if account["account_number"] == int(from_account_number):
            account["account_balance"] -= amount

        if account["account_number"] == int(to_account_number):
            account["account_balance"] += amount

    return "New account balance for account " + str(from_account_number) + " is: " + str(get_account_balance(from_account_number))

## Describing the functions in the GPT schema

Second, we define each of the possible functions the model can leverage to fulfill customer banking requests. It is important to describe each of the method as clear and complete as possible

In [28]:
functions= [
    {
        "name": "create_bank_account",
        "description": "Creates a new bank account. Can also be called: Open a new bank account, Create an account",
        "parameters": {
            "type": "object",
            "properties": {
                "account_type": {
                    "type": "string",
                    "enum": ["checkings", "savings"],
                    "description": "The account type. Can be either checkings or savings. You must ask the user if not provided.",
                },
                "account_balance": {
                    "type": "number",
                    "description": "The amount of money used to open the account. You must ask the user if not provided."
                },
                "user_id": {
                    "type": "string",
                    "description": "The user id of the user opening the account. You must ask the user if not provided."
                }
            },
            "required": ["account_type", "account_balance", "user_id"],
        },
    },
    {
        "name": "get_user_accounts",
        "description": "Returns a list of accounts that belong to the given user_id. Can also be called: Get my accounts, Get my bank accounts, list my accounts",
        "parameters": {
            "type": "object",
            "properties": {
                "user_id": {
                    "type": "string",
                    "description": "The user id owner of the accounts. You must ask the user if not provided." ,
                }
            },
        },
        "required": ["user_id"],
    },
    {
        "name": "get_account_balance",
        "description": "Returns the account balance for the given account_number. Can also be called: Get my account balance, Get my balance",
        "parameters": {
            "type": "object",
            "properties": {
                "account_number": {
                    "type": "string",
                    "description": "The account number of the account. You must ask the user if not provided." ,
                }
            },
        },
        "required": ["account_number"],
    },
    {
        "name": "make_deposit",
        "description": "Makes a deposit into the account with the given account_number. Can also be called: Deposit money, Add money to my account",
        "parameters": {
            "type": "object",
            "properties": {
                "account_number": {
                    "type": "string",
                    "description": "The account number of the account. You must ask the user if not provided." ,
                },
                "amount": {
                    "type": "number",
                    "description": "The amount of money to deposit. You must ask the user if not provided." ,
                }
            },
        },
        "required": ["account_number", "amount"],
    },
    {
        "name": "make_transfer_between_accounts",
        "description": "Makes a transfer between the two accounts. Can also be called: Transfer money, Send money, Send money to my friend",
        "parameters": {
            "type": "object",
            "properties": {
                "from_account_number": {
                    "type": "string",
                    "description": "The account number of the account to transfer from. You must ask the user if not provided." ,
                },
                "to_account_number": {
                    "type": "string",
                    "description": "The account number of the account to transfer to. You must ask the user if not provided." ,
                },
                "amount": {
                    "type": "number",
                    "description": "The amount of money to transfer. You must ask the user if not provided." ,
                }
            },
        },
        "required": ["from_account_number", "to_account_number", "amount"],
    }

]

available_functions = {
            "create_bank_account": create_bank_account,
            "get_user_accounts": get_user_accounts,
            "get_account_balance": get_account_balance,
            "make_deposit": make_deposit,
            "make_transfer_between_accounts": make_transfer_between_accounts
        } 

In [29]:
import inspect

# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided 
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

## Calling the OpenAI API

We create the `run_conversation` method that will call the OpenAI API and check if GPT wanted to call a function. If so, the code will execute the call against the recommended function, gather the response and call GPT for a second time with the functions response as part of the prompt history.

In [30]:
def run_conversation(messages, functions, available_functions, deployment_id):
    # Step 1: send the conversation and available functions to GPT

    response = openai.ChatCompletion.create(
        deployment_id=deployment_id,
        messages=messages,
        functions=functions,
        function_call="auto",
        temperature=0.0
    )
    response_message = response["choices"][0]["message"]
    print("Response from GPT:")
    print(response_message)

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        print("Recommended Function call:")
        print(response_message.get("function_call"))
        print()
        
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        
        function_name = response_message["function_call"]["name"]
        
        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name]  
        
        # verify function has correct number of arguments
        function_args = json.loads(response_message["function_call"]["arguments"])

        if check_args(function_to_call, function_args) is False:
            print("Invalid number of arguments for function: " + function_name)
            return "Invalid number of arguments for function: " + function_name
        function_response = function_to_call(**function_args)
        
        print("Output of function call:")
        print(function_response)
        print()
        
        # Step 4: send the info on the function call and function response to GPT
        
        # adding assistant response to messages
        messages.append(
            {
                "role": response_message["role"],
                "function_call": {
                    "name": response_message["function_call"]["name"],
                    "arguments": response_message["function_call"]["arguments"],
                },
                "content": None
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                # add a content property to the function response converted to string  so that it can be used by GPT
                "content": str(function_response)
            }
        )  # extend conversation with function response

        print("Messages in second request:")
        for message in messages:
            print(message)
        print()

        second_response = openai.ChatCompletion.create(
            messages=messages,
            deployment_id=deployment_id
        )  # get a new response from GPT where it can see the function response

        return second_response
    return response

### Example on how all these pieces fit together



In [31]:
messages = []

messages.append({"role": "system",   "content": "You're a Retail Bank AI assistant for Contozo Durable Bank, designed to help users open and manage their accounts."})
messages.append({"role": "system",   "content": "When a user asks for opening a new account, you should call the create_bank_account function"})
messages.append({"role": "system",   "content": "Do not make assumptions about what values to use with functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "system",   "content": "Don't recommend function calls if the user has not provided the arguments of the function."})
messages.append({"role": "system",   "content": "If the user does not provide the arguments of a function to call, do not assume them. Ask the user for the values."})
messages.append({"role": "system",   "content": "DO NOT assign default or empty values to the arguments of the functions."})
messages.append({"role":"user",      "content": "I want to open an account!"})
messages.append({"role":"assistant", "content": "Sure, I can help you with that. Could you please provide me with the following information:\n\n1. Account type: Is it a savings account or a checking account?\n2. Initial deposit amount: How much money would you like to deposit to open the account?\n3. User ID: Do you have a user ID with us? If not, we can NOT create one for you.\n\nPlease provide the above information so that I can assist you in opening an account."})

messages.append({"role": "user", "content": "I want to open an account"})

assistant_response = run_conversation(messages, functions, available_functions, GPT_ENGINE)

# check if assistant_response["choices"] is not null
if assistant_response["choices"]:
    print("Assistant Response:")
    print(assistant_response["choices"][0]["message"])


Response from GPT:
{
  "role": "assistant",
  "content": "Sure, I can help you with that. Could you please provide me with the following information:\n\n1. Account type: Is it a savings account or a checking account?\n2. Initial deposit amount: How much money would you like to deposit to open the account?\n3. User ID: Do you have a user ID with us? If not, we can create one for you.\n\nPlease provide the above information so that I can assist you in opening an account."
}
Assistant Response:
{
  "role": "assistant",
  "content": "Sure, I can help you with that. Could you please provide me with the following information:\n\n1. Account type: Is it a savings account or a checking account?\n2. Initial deposit amount: How much money would you like to deposit to open the account?\n3. User ID: Do you have a user ID with us? If not, we can create one for you.\n\nPlease provide the above information so that I can assist you in opening an account."
}


In [32]:
messages.append(assistant_response["choices"][0]["message"])

messages.append({"role": "user", "content": "I want to open a checking account with 100 dollars, my user id is nico1234"})

assistant_response = run_conversation(messages, functions, available_functions, GPT_ENGINE)

print(assistant_response["choices"][0]["message"])

Response from GPT:
{
  "role": "assistant",
  "function_call": {
    "name": "create_bank_account",
    "arguments": "{\n  \"account_type\": \"checkings\",\n  \"account_balance\": 100,\n  \"user_id\": \"nico1234\"\n}"
  }
}
Recommended Function call:
{
  "name": "create_bank_account",
  "arguments": "{\n  \"account_type\": \"checkings\",\n  \"account_balance\": 100,\n  \"user_id\": \"nico1234\"\n}"
}

Output of function call:
{'account_number': 7969976976, 'account_type': 'checkings', 'account_balance': 100, 'user_id': 'nico1234'}

Messages in second request:
{'role': 'system', 'content': "You're a Retail Bank AI assistant for Contozo Durable Bank, designed to help users open and manage their accounts."}
{'role': 'system', 'content': 'When a user asks for opening a new account, you should call the create_bank_account function'}
{'role': 'system', 'content': 'Do not make assumptions about what values to use with functions. Ask for clarification if a user request is ambiguous.'}
{'role':

In [39]:
accounts


[{'account_number': 7969976976,
  'account_type': 'checkings',
  'account_balance': 123,
  'user_id': 'nico1234'},
 {'account_number': 9732778002,
  'account_type': 'savings',
  'account_balance': 210,
  'user_id': 'nico1234'}]

In [34]:
messages.append(assistant_response["choices"][0]["message"])

messages.append({"role": "user", "content": "I want to open a savings account with 200 dollars, my user id is nico1234"})

assistant_response = run_conversation(messages, functions, available_functions, GPT_ENGINE)

print(assistant_response["choices"][0]["message"])

Response from GPT:
{
  "role": "assistant",
  "function_call": {
    "name": "create_bank_account",
    "arguments": "{\n  \"account_type\": \"savings\",\n  \"account_balance\": 200,\n  \"user_id\": \"nico1234\"\n}"
  }
}
Recommended Function call:
{
  "name": "create_bank_account",
  "arguments": "{\n  \"account_type\": \"savings\",\n  \"account_balance\": 200,\n  \"user_id\": \"nico1234\"\n}"
}

Output of function call:
{'account_number': 9732778002, 'account_type': 'savings', 'account_balance': 200, 'user_id': 'nico1234'}

Messages in second request:
{'role': 'system', 'content': "You're a Retail Bank AI assistant for Contozo Durable Bank, designed to help users open and manage their accounts."}
{'role': 'system', 'content': 'When a user asks for opening a new account, you should call the create_bank_account function'}
{'role': 'system', 'content': 'Do not make assumptions about what values to use with functions. Ask for clarification if a user request is ambiguous.'}
{'role': 'syst

In [35]:

messages.append(assistant_response["choices"][0]["message"])

messages.append({"role": "user", "content": "I want to list my accounts, my user id is nico1234"})
assistant_response = run_conversation(messages, functions, available_functions, GPT_ENGINE) 

print(assistant_response["choices"][0]["message"])


Response from GPT:
{
  "role": "assistant",
  "function_call": {
    "name": "get_user_accounts",
    "arguments": "{\n  \"user_id\": \"nico1234\"\n}"
  }
}
Recommended Function call:
{
  "name": "get_user_accounts",
  "arguments": "{\n  \"user_id\": \"nico1234\"\n}"
}

Output of function call:
[{'account_number': 7969976976, 'account_type': 'checkings', 'account_balance': 100, 'user_id': 'nico1234'}, {'account_number': 9732778002, 'account_type': 'savings', 'account_balance': 200, 'user_id': 'nico1234'}]

Messages in second request:
{'role': 'system', 'content': "You're a Retail Bank AI assistant for Contozo Durable Bank, designed to help users open and manage their accounts."}
{'role': 'system', 'content': 'When a user asks for opening a new account, you should call the create_bank_account function'}
{'role': 'system', 'content': 'Do not make assumptions about what values to use with functions. Ask for clarification if a user request is ambiguous.'}
{'role': 'system', 'content': "Don

In [37]:
messages.append(assistant_response["choices"][0]["message"])
messages.append({"role": "user", "content": "I want to deposit $33 dollars to my account 7969976976"})
assistant_response = run_conversation(messages, functions, available_functions, GPT_ENGINE)

print(assistant_response["choices"][0]["message"])

Response from GPT:
{
  "role": "assistant",
  "function_call": {
    "name": "make_deposit",
    "arguments": "{\n  \"account_number\": \"7969976976\",\n  \"amount\": 33\n}"
  }
}
Recommended Function call:
{
  "name": "make_deposit",
  "arguments": "{\n  \"account_number\": \"7969976976\",\n  \"amount\": 33\n}"
}

make_deposit
7969976976 33
Output of function call:
133

Messages in second request:
{'role': 'system', 'content': "You're a Retail Bank AI assistant for Contozo Durable Bank, designed to help users open and manage their accounts."}
{'role': 'system', 'content': 'When a user asks for opening a new account, you should call the create_bank_account function'}
{'role': 'system', 'content': 'Do not make assumptions about what values to use with functions. Ask for clarification if a user request is ambiguous.'}
{'role': 'system', 'content': "Don't recommend function calls if the user has not provided the arguments of the function."}
{'role': 'system', 'content': 'If the user does 

In [38]:
messages.append(assistant_response["choices"][0]["message"])

messages.append({"role": "user", "content": "Transfer $10 from account 7969976976 to account 9732778002"})
assistant_response = run_conversation(messages, functions, available_functions, GPT_ENGINE)

print(assistant_response["choices"][0]["message"])

Response from GPT:
{
  "role": "assistant",
  "function_call": {
    "name": "make_transfer_between_accounts",
    "arguments": "{\n  \"from_account_number\": \"7969976976\",\n  \"to_account_number\": \"9732778002\",\n  \"amount\": 10\n}"
  }
}
Recommended Function call:
{
  "name": "make_transfer_between_accounts",
  "arguments": "{\n  \"from_account_number\": \"7969976976\",\n  \"to_account_number\": \"9732778002\",\n  \"amount\": 10\n}"
}

make_transfer_between_accounts
7969976976 9732778002 10
Output of function call:
New account balance for account 7969976976 is: 123

Messages in second request:
{'role': 'system', 'content': "You're a Retail Bank AI assistant for Contozo Durable Bank, designed to help users open and manage their accounts."}
{'role': 'system', 'content': 'When a user asks for opening a new account, you should call the create_bank_account function'}
{'role': 'system', 'content': 'Do not make assumptions about what values to use with functions. Ask for clarification 

In [48]:
pwd

'/Users/nicoj/cserepos/anaconda'